# **SP23-BAI-046**
# **NOOR FATIMA **

In [25]:
from google.colab import files
uploaded = files.upload()

Saving animals.zip to animals.zip


In [26]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping

In [27]:
if not os.path.exists("animals.zip"):
  print("Error:Directory not found")
else:
  print("Directory found")

Directory found


In [28]:
import zipfile
with zipfile.ZipFile("animals.zip", 'r') as zip_ref:
    zip_ref.extractall("animals_dataset")

Data Generators

In [33]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [44]:
train_generator = tf.keras.utils.image_dataset_from_directory(
    "animals_dataset",
    image_size=(150, 150),
    batch_size=64,
    validation_split=0.2,
    seed=123,
    subset='training',
    label_mode='categorical',
    shuffle=True
)

val_generator = tf.keras.utils.image_dataset_from_directory(
    "animals_dataset",
    image_size=(150, 150),
    batch_size=64,
    seed=123,
    validation_split=0.2,
    subset='validation',
    label_mode='categorical',
    shuffle=True
)

Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 600 files for validation.


In [45]:
classes_names = train_generator.class_names
print(classes_names)

['cats', 'dogs', 'panda']


In [46]:
AUTOTUNE = tf.data.AUTOTUNE

train_generator = train_generator.cache().prefetch(buffer_size=AUTOTUNE)
val_generator = val_generator.cache().prefetch(buffer_size=AUTOTUNE)

Simple model

In [47]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [49]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 105s 3s/step - accuracy: 0.4291 - loss: 408.3094 - val_accuracy: 0.5133 - val_loss: 1.0025
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.6236 - loss: 0.8674 - val_accuracy: 0.5717 - val_loss: 0.9260
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.7436 - loss: 0.6868 - val_accuracy: 0.5850 - val_loss: 0.9191
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 100s 3s/step - accuracy: 0.8342 - loss: 0.4664 - val_accuracy: 0.5833 - val_loss: 1.0078
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.9085 - loss: 0.3133 - val_accuracy: 0.5967 - val_loss: 1.2807
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 99s 3s/step - accuracy: 0.9180 - loss: 0.3158 - val_accuracy: 0.5500 - val_loss: 1.4897
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 0.9347 - loss: 0.2047 - val_accuracy: 0.5600 - val_loss: 2.2996
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.9531 - loss: 0.1333 - val_accuracy: 0.5783 - val_

In [50]:
model_loss, model_acc= model.evaluate(val_generator)
print("Simple model accuracy: ", model_acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5582 - loss: 2.9883
Simple model accuracy:  0.5649999976158142


Model with regularization

In [51]:
regularized_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])


In [52]:
regularized_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [54]:
regularized_history = regularized_model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=[early_stopping])

Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 153s 4s/step - accuracy: 0.3895 - loss: 29.7908 - val_accuracy: 0.5333 - val_loss: 6.3296
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 134s 4s/step - accuracy: 0.4363 - loss: 6.3332 - val_accuracy: 0.5833 - val_loss: 5.8229
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 147s 4s/step - accuracy: 0.5147 - loss: 5.8609 - val_accuracy: 0.5933 - val_loss: 5.5390
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 145s 4s/step - accuracy: 0.5390 - loss: 5.5818 - val_accuracy: 0.5983 - val_loss: 5.3533
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.5371 - loss: 5.3864 - val_accuracy: 0.6133 - val_loss: 5.1376


In [55]:
regularized_loss, regularized_acc= model.evaluate(val_generator)
print("Regularized  model accuracy: ", regularized_acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5582 - loss: 2.9883
Regularized  model accuracy:  0.5649999976158142


In [ ]:
rmsprop_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

In [ ]:
rmsprop_model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
rmsprop_history = rmsprop_model.fit(train_generator,  epochs=3, validation_data=val_generator)

In [ ]:
rms_loss, rms_acc= model.evaluate(val_generator)
print("RMSprop model accuracy: ", rms_acc)

# **Plotting Results**

Base model

In [ ]:
plt.figure(figsize=(12, 4))
plt.suptitle("Initial Model")
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

Model with regulariation

In [ ]:
plt.figure(figsize=(12, 4))
plt.suptitle("Regularized Model")
plt.subplot(1, 2, 1)
plt.plot(regularized_history.history['accuracy'], label='Train Accuracy')
plt.plot(regularized_history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(regularized_history.history['loss'], label='Train Loss')
plt.plot(regularized_history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

Model with RMSprop

In [ ]:
plt.figure(figsize=(12, 4))
plt.suptitle("RMSprop Model")
plt.subplot(1, 2, 1)
plt.plot(rmsprop_history.history['accuracy'], label='Train Accuracy')
plt.plot(rmsprop_history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(rmsprop_history.history['loss'], label='Train Loss')
plt.plot(rmsprop_history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

# **Summary**

A Convolutional Neural Network (CNN) was developed to classify images of animals into three categories. The initial model, consisting of two convolutional layers followed by dense layers, achieved decent accuracy but showed early signs of overfitting. To improve its performance, several techniques were applied, including L2 regularization, dropout layers, and the EarlyStopping callback. Additionally, the optimizer was tuned by lowering the learning rate for Adam and comparing it with RMSprop. Among the models tested, the regularized model with dropout and early stopping performed the best, showing improved validation accuracy and better generalization to unseen data. In contrast, the RMSprop model was effective but did not outperform the regularized model. Overall, the experiment demonstrated that using regularization techniques and fine-tuning training strategies can significantly enhance model performance and prevent overfitting.